# Stone Soup Tracking Algorithm Comparison

Visualize and compare openpilot's KF1D filter against Stone Soup's Kalman variants,
multi-target trackers, and fusion methods.

**Prerequisites**: `pip install stonesoup matplotlib`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

## 1. Filter Comparison: KF1D vs Stone Soup Kalman Variants

Compare openpilot's production KF1D against Kalman, Extended Kalman,
Unscented Kalman, Cubature Kalman, and Particle filters.

In [ ]:
from openpilot.tools.stonesoup.comparison import (
    create_constant_velocity_scenario,
    compare_filters,
    format_comparison_report,
)

scenario = create_constant_velocity_scenario(dt=0.05, duration=10.0)
metrics = compare_filters(scenario)
print(format_comparison_report(scenario, metrics))

In [ ]:
# Plot position estimates vs ground truth
fig, axes = plt.subplots(2, 1, sharex=True)

times = [r.timestamp for r in scenario.results['KF1D']]
gt_pos = [r.ground_truth[0] for r in scenario.results['KF1D']]
measurements = [r.measurement for r in scenario.results['KF1D'] if r.measurement is not None]
meas_times = [r.timestamp for r in scenario.results['KF1D'] if r.measurement is not None]

axes[0].plot(times, gt_pos, 'k--', label='Ground Truth', linewidth=2)
axes[0].scatter(meas_times, measurements, c='gray', s=10, alpha=0.5, label='Measurements')

for name, results in scenario.results.items():
    pos = [r.state[0] for r in results]
    axes[0].plot(times, pos, label=name, alpha=0.8)

axes[0].set_ylabel('Position (m)')
axes[0].set_title('Filter Position Estimates')
axes[0].legend(fontsize=8, ncol=3)

# Plot position error
for name, results in scenario.results.items():
    errors = [abs(r.state[0] - r.ground_truth[0]) for r in results]
    axes[1].plot(times, errors, label=name, alpha=0.8)

axes[1].set_ylabel('Position Error (m)')
axes[1].set_xlabel('Time (s)')
axes[1].set_title('Absolute Position Error')
axes[1].legend(fontsize=8, ncol=3)

plt.tight_layout()
plt.show()

In [ ]:
# RMSE comparison bar chart
names = [m.filter_name for m in metrics]
rmse_pos = [m.rmse_position for m in metrics]
rmse_vel = [m.rmse_velocity for m in metrics]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

colors = plt.cm.Set2(np.linspace(0, 1, len(names)))
axes[0].bar(names, rmse_pos, color=colors)
axes[0].set_ylabel('RMSE (m)')
axes[0].set_title('Position RMSE by Filter')
axes[0].tick_params(axis='x', rotation=30)

axes[1].bar(names, rmse_vel, color=colors)
axes[1].set_ylabel('RMSE (m/s)')
axes[1].set_title('Velocity RMSE by Filter')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

## 2. Multi-Target Tracking

Compare JPDA and GNN trackers on highway scenarios with MOTA/MOTP metrics.

In [ ]:
from openpilot.tools.stonesoup.multi_target import (
    create_highway_scenario,
    compare_multi_target_trackers,
    format_tracking_report,
)

mt_scenario = create_highway_scenario(dt=0.1, duration=10.0, n_vehicles=4)
mt_metrics = compare_multi_target_trackers(mt_scenario)
print(format_tracking_report(mt_scenario, mt_metrics))

In [ ]:
# Multi-target tracking metrics comparison
tracker_names = list(mt_metrics.keys())
mota_vals = [mt_metrics[n].mota for n in tracker_names]
motp_vals = [mt_metrics[n].motp for n in tracker_names]
id_switches = [mt_metrics[n].id_switches for n in tracker_names]

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

axes[0].bar(tracker_names, mota_vals, color=['steelblue', 'coral'])
axes[0].set_ylabel('MOTA')
axes[0].set_title('Multi-Object Tracking Accuracy')
axes[0].set_ylim(0, 1)

axes[1].bar(tracker_names, motp_vals, color=['steelblue', 'coral'])
axes[1].set_ylabel('MOTP (m)')
axes[1].set_title('Multi-Object Tracking Precision')

axes[2].bar(tracker_names, id_switches, color=['steelblue', 'coral'])
axes[2].set_ylabel('Count')
axes[2].set_title('ID Switches')

plt.tight_layout()
plt.show()

## 3. Track Fusion

Compare Covariance Intersection fusion against single-sensor baselines.

In [ ]:
from openpilot.tools.stonesoup.track_fusion import (
    create_fusion_scenario,
    compare_fusion_methods,
    format_fusion_report,
)

fusion_scenario = create_fusion_scenario(dt=0.05, duration=5.0)
fusion_metrics = compare_fusion_methods(fusion_scenario)
print(format_fusion_report(fusion_scenario, fusion_metrics))

In [ ]:
# Fusion RMSE comparison
fusion_names = list(fusion_metrics.keys())
fusion_rmse = [fusion_metrics[n].rmse_position for n in fusion_names]

fig, ax = plt.subplots(figsize=(8, 5))
colors = ['steelblue', 'coral', 'mediumseagreen', 'goldenrod'][:len(fusion_names)]
ax.bar(fusion_names, fusion_rmse, color=colors)
ax.set_ylabel('Position RMSE (m)')
ax.set_title('Fusion Method Comparison')
ax.tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.show()

## 4. Benchmark Scenarios

Run all benchmark scenarios and visualize results.

In [ ]:
from openpilot.tools.stonesoup.benchmarks.scenarios import (
    create_highway_scenario as bm_highway,
    create_cut_in_scenario,
    create_cut_out_scenario,
    create_multi_vehicle_scenario,
    create_occlusion_scenario,
    create_noisy_scenario,
)

scenarios = {
    'Highway': bm_highway(),
    'Cut-in': create_cut_in_scenario(),
    'Cut-out': create_cut_out_scenario(),
    'Multi-vehicle': create_multi_vehicle_scenario(),
    'Occlusion': create_occlusion_scenario(),
    'Noisy (weather)': create_noisy_scenario(),
}

print(f'Loaded {len(scenarios)} benchmark scenarios:')
for name, s in scenarios.items():
    n_steps = len(s.ground_truth) if hasattr(s, 'ground_truth') else 'N/A'
    print(f'  {name}: {n_steps} timesteps')

In [ ]:
# Visualize scenario trajectories
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for ax, (name, s) in zip(axes.flat, scenarios.items()):
    if hasattr(s, 'ground_truth'):
        gt = s.ground_truth
        if hasattr(gt[0], 'x'):
            x = [p.x for p in gt]
            y = [p.y for p in gt]
            ax.plot(x, y, 'k-', linewidth=2)
            ax.set_xlabel('x (m)')
            ax.set_ylabel('y (m)')
        else:
            times = range(len(gt))
            ax.plot(times, gt, 'k-', linewidth=2)
            ax.set_xlabel('Timestep')
            ax.set_ylabel('Value')
    ax.set_title(name)
    ax.grid(True, alpha=0.3)

plt.suptitle('Benchmark Scenario Ground Truth Trajectories', fontsize=14)
plt.tight_layout()
plt.show()